# Graph Coloring

In [2]:
from datetime import datetime

def LogInfo(msg):
    print(datetime.now().strftime('%H:%M:%S') + ' - ' + msg)

def ReadFile(file_location):
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
    return input_data

def GetInputs(input_data):
    input_data = input_data.splitlines()
    input_data = [list(map(int, x.split(' ')))  for x in input_data]
    inputs={}
    inputs['nbnodes'] = input_data[0][0]
    inputs['nbedges'] = input_data[0][1]
    inputs['startnodes'] = [row[0] for row in input_data[1:]]
    inputs['endnodes'] = [row[1] for row in input_data[1:]]
    adjacentnodes = {}
    for row in input_data[1:]:
        node1 = row[0]
        node2 = row[1]
        if node1 not in adjacentnodes:
            adjacentnodes[node1] = []
        if node2 not in adjacentnodes[node1]:
            adjacentnodes[node1].append(node2)
        if node2 not in adjacentnodes:
            adjacentnodes[node2] = []
        if node1 not in adjacentnodes[node2]:
            adjacentnodes[node2].append(node1)
    for k, v in adjacentnodes.items():
        v.sort()
    inputs['nodes'] = list(adjacentnodes.keys())
    inputs['colors'] = range(len(inputs['nodes'])) # initialize colors to one for each node
    inputs['adjacentnodes'] = adjacentnodes
    return inputs

inputs = GetInputs(ReadFile('data/gc_20_1'))
#inputs

### Option 1: Greedy algorithm

In [3]:
# %%time

# function to run the algorithm for a given, ordered, list of nodes 
def RunGreedyAlgorithm(inputNodes, inputs):
    nodecolors = {}

    # loop on each node
    for node in inputs['nodes']:
        #print('>> Node ', node)
        for color in inputs['colors']:
            #print('    >> Color ', color)
            # try to assign each color until one is valid
            # to determine if a color is valid, we look at all adjacent nodes and discard already assigned colors.
            isAvailable = True

            if node not in inputs['adjacentnodes']:
                # if the node doesn't have any adjacent nodes, assign the first available color
                print('does not have any neighboors')
                nodecolors[node] = color
                break

            adjacentNodes = inputs['adjacentnodes'][node]
            for adjacentNode in adjacentNodes:
                if adjacentNode in nodecolors:
                    # if the node already has a color assigned ...
                    if nodecolors[adjacentNode] == color:
                        # ... and if the color assigned is the same than the current one, discard it.
                        isAvailable = False
                        break

            if isAvailable:
                nodecolors[node] = color
                break

    output = {}
    output['objective'] = len(set(nodecolors.values()))
    output['variables'] = list(nodecolors.values())
    return output

# try different orders of input nodes
# inputs['nbnodes']
tmpInput = inputs['nodes'].copy()
bestOutput = {}
worstOutput = {}
for i in range(2):
#for i in range(inputs['nbnodes']):
    tmpInput = tmpInput[-1:] + tmpInput[:-1]
    tmpOutput = RunGreedyAlgorithm(tmpInput, inputs)
    if bestOutput == {}:
        bestOutput = tmpOutput
    elif tmpOutput['objective'] < bestOutput['objective']:
        bestOutput = tmpOutput
    if worstOutput == {}:
        worstOutput = tmpOutput
    elif worstOutput['objective'] < tmpOutput['objective']:
        worstOutput = tmpOutput  
    LogInfo('Configuration ' + str(i) + ' computed; objective = ' + str(tmpOutput['objective']))

LogInfo('Results:')
LogInfo('Best output: ' + str(bestOutput['objective']))
LogInfo('Worst output: ' + str(worstOutput['objective']))

21:10:48 - Configuration 0 computed; objective = 3
21:10:48 - Configuration 1 computed; objective = 3
21:10:48 - Results:
21:10:48 - Best output: 3
21:10:48 - Worst output: 3


### Option 2: OR Tools - CP

In [4]:
from ortools.constraint_solver import pywrapcp

# Creates the solver
solver = pywrapcp.Solver("GraphColoring")

# run greedy algorithm first, to find an upper bound on the number of required colors
greedyoutputs = RunGreedyAlgorithm(inputs['nodes'], inputs)
print('Greedy objective: ', greedyoutputs['objective'])

# Creates the variables
variables = []
for i in range(inputs['nbnodes']):
    # x = solver.IntVar(0, inputs['nbnodes'] - 1, "x" + str(i))
    x = solver.IntVar(0, greedyoutputs['objective'] - 1, "x" + str(i))
    variables.append(x)

# Create the constraints
for edge in range(inputs['nbedges']):
    # get variables corresponding to start and end nodes
    snode = inputs['startnodes'][edge]
    enode = inputs['endnodes'][edge]
    #print('Adding constraint: ', snode, ' != ', enode)
    xs = variables[snode]
    xe = variables[enode]
    solver.Add(xs != xe)

# Create the decision builder
db = solver.Phase(variables, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)
solver.Solve(db)

count = 0
outputs = {}
while solver.NextSolution():
    count += 1
    # get variable values
    results = []
    for x in variables:
        results.append(x.Value())
    objective = len(set(results))
    if outputs == {}:
        outputs['objective'] = objective
        outputs['results'] = results
    elif objective < outputs['objective']:
        outputs['objective'] = objective
        outputs['results'] = results
    if (count % 100) == 0:
        print("Solution", count, '; objective=', objective)

print(count, ' solutions scanned.')
print('objective: ', outputs['objective'])
print('results: ', outputs['results'])

Greedy objective:  3
Solution 100 ; objective= 3
Solution 200 ; objective= 3
Solution 300 ; objective= 3
Solution 400 ; objective= 3
Solution 500 ; objective= 3
Solution 600 ; objective= 3
Solution 700 ; objective= 3
Solution 800 ; objective= 3
Solution 900 ; objective= 3
Solution 1000 ; objective= 3
Solution 1100 ; objective= 3
Solution 1200 ; objective= 3
Solution 1300 ; objective= 3
Solution 1400 ; objective= 3
Solution 1500 ; objective= 3
Solution 1600 ; objective= 3
Solution 1700 ; objective= 3
Solution 1800 ; objective= 3
Solution 1900 ; objective= 3
Solution 2000 ; objective= 3
Solution 2100 ; objective= 3
Solution 2200 ; objective= 3
Solution 2300 ; objective= 3
Solution 2400 ; objective= 3
Solution 2500 ; objective= 3
Solution 2600 ; objective= 3
Solution 2700 ; objective= 3
Solution 2800 ; objective= 3
Solution 2900 ; objective= 3
Solution 3000 ; objective= 3
Solution 3100 ; objective= 3
Solution 3200 ; objective= 3
Solution 3300 ; objective= 3
Solution 3400 ; objective= 3
So

Solution 40200 ; objective= 3
Solution 40300 ; objective= 3
Solution 40400 ; objective= 3
Solution 40500 ; objective= 3
Solution 40600 ; objective= 3
Solution 40700 ; objective= 3
Solution 40800 ; objective= 3
Solution 40900 ; objective= 3
Solution 41000 ; objective= 3
Solution 41100 ; objective= 3
Solution 41200 ; objective= 3
Solution 41300 ; objective= 3
Solution 41400 ; objective= 3
Solution 41500 ; objective= 3
Solution 41600 ; objective= 3
Solution 41700 ; objective= 3
Solution 41800 ; objective= 3
Solution 41900 ; objective= 3
Solution 42000 ; objective= 3
Solution 42100 ; objective= 3
Solution 42200 ; objective= 3
Solution 42300 ; objective= 3
Solution 42400 ; objective= 3
Solution 42500 ; objective= 3
Solution 42600 ; objective= 3
Solution 42700 ; objective= 3
Solution 42800 ; objective= 3
Solution 42900 ; objective= 3
Solution 43000 ; objective= 3
Solution 43100 ; objective= 3
Solution 43200 ; objective= 3
Solution 43300 ; objective= 3
Solution 43400 ; objective= 3
Solution 4

Solution 86700 ; objective= 3
Solution 86800 ; objective= 3
Solution 86900 ; objective= 3
Solution 87000 ; objective= 3
Solution 87100 ; objective= 3
Solution 87200 ; objective= 3
Solution 87300 ; objective= 3
Solution 87400 ; objective= 3
Solution 87500 ; objective= 3
Solution 87600 ; objective= 3
Solution 87700 ; objective= 3
Solution 87800 ; objective= 3
Solution 87900 ; objective= 3
Solution 88000 ; objective= 3
Solution 88100 ; objective= 3
Solution 88200 ; objective= 3
Solution 88300 ; objective= 3
Solution 88400 ; objective= 3
Solution 88500 ; objective= 3
Solution 88600 ; objective= 3
Solution 88700 ; objective= 3
Solution 88800 ; objective= 3
Solution 88900 ; objective= 3
Solution 89000 ; objective= 3
Solution 89100 ; objective= 3
Solution 89200 ; objective= 3
Solution 89300 ; objective= 3
Solution 89400 ; objective= 3
Solution 89500 ; objective= 3
Solution 89600 ; objective= 3
Solution 89700 ; objective= 3
Solution 89800 ; objective= 3
Solution 89900 ; objective= 3
Solution 9

Solution 131600 ; objective= 3
Solution 131700 ; objective= 3
Solution 131800 ; objective= 3
Solution 131900 ; objective= 3
Solution 132000 ; objective= 3
Solution 132100 ; objective= 3
Solution 132200 ; objective= 3
Solution 132300 ; objective= 3
Solution 132400 ; objective= 3
Solution 132500 ; objective= 3
Solution 132600 ; objective= 3
Solution 132700 ; objective= 3
Solution 132800 ; objective= 3
Solution 132900 ; objective= 3
Solution 133000 ; objective= 3
Solution 133100 ; objective= 3
Solution 133200 ; objective= 3
Solution 133300 ; objective= 3
Solution 133400 ; objective= 3
Solution 133500 ; objective= 3
Solution 133600 ; objective= 3
Solution 133700 ; objective= 3
Solution 133800 ; objective= 3
Solution 133900 ; objective= 3
Solution 134000 ; objective= 3
Solution 134100 ; objective= 3
Solution 134200 ; objective= 3
Solution 134300 ; objective= 3
Solution 134400 ; objective= 3
Solution 134500 ; objective= 3
Solution 134600 ; objective= 3
Solution 134700 ; objective= 3
Solution

Solution 174200 ; objective= 3
Solution 174300 ; objective= 3
Solution 174400 ; objective= 3
Solution 174500 ; objective= 3
Solution 174600 ; objective= 3
Solution 174700 ; objective= 3
Solution 174800 ; objective= 3
Solution 174900 ; objective= 3
Solution 175000 ; objective= 3
Solution 175100 ; objective= 3
Solution 175200 ; objective= 3
Solution 175300 ; objective= 3
Solution 175400 ; objective= 3
Solution 175500 ; objective= 3
Solution 175600 ; objective= 3
Solution 175700 ; objective= 3
Solution 175800 ; objective= 3
Solution 175900 ; objective= 3
Solution 176000 ; objective= 3
Solution 176100 ; objective= 3
Solution 176200 ; objective= 3
Solution 176300 ; objective= 3
Solution 176400 ; objective= 3
Solution 176500 ; objective= 3
Solution 176600 ; objective= 3
Solution 176700 ; objective= 3
Solution 176800 ; objective= 3
Solution 176900 ; objective= 3
Solution 177000 ; objective= 3
Solution 177100 ; objective= 3
Solution 177200 ; objective= 3
Solution 177300 ; objective= 3
Solution

Solution 224100 ; objective= 3
Solution 224200 ; objective= 3
Solution 224300 ; objective= 3
Solution 224400 ; objective= 3
Solution 224500 ; objective= 3
Solution 224600 ; objective= 3
Solution 224700 ; objective= 3
Solution 224800 ; objective= 3
Solution 224900 ; objective= 3
Solution 225000 ; objective= 3
Solution 225100 ; objective= 3
Solution 225200 ; objective= 3
Solution 225300 ; objective= 3
Solution 225400 ; objective= 3
Solution 225500 ; objective= 3
Solution 225600 ; objective= 3
Solution 225700 ; objective= 3
Solution 225800 ; objective= 3
Solution 225900 ; objective= 3
Solution 226000 ; objective= 3
Solution 226100 ; objective= 3
Solution 226200 ; objective= 3
Solution 226300 ; objective= 3
Solution 226400 ; objective= 3
Solution 226500 ; objective= 3
Solution 226600 ; objective= 3
Solution 226700 ; objective= 3
Solution 226800 ; objective= 3
Solution 226900 ; objective= 3
Solution 227000 ; objective= 3
Solution 227100 ; objective= 3
Solution 227200 ; objective= 3
Solution